# AVL树
- 最早的可以自平衡的二叉树
- 什么是平衡？对于任意一个节点，左子树和右子树的高度差不超过1
- 本节代码以二分搜索树的代码结构作为模板
- 本节代码和bobo老师的代码架构有所不同，只是将维护平衡性的代码封装了一下，这样removeMin/removeMax这些成员函数就能轻易的实现了，保证接口一致性的同时以前的二分搜索树的删除节点的代码也不用动了。

In [1]:
import numpy as np
from collections import deque
import random

In [2]:
class Node:
    def __init__(self, elem):
        """
        AVL树中节点类的构造函数，就新增了一个成员属性：height
        代表了当前节点的高度，默认值为1
        """
        self.elem = elem
        self.left = self.right = None
        self.height = 1

In [3]:
class AvlTree:
    def __init__(self):
        """二分搜索树的构造函数"""
        self.root = None
        self.size = 0
    
    def isBst(self):
        """
        检验当前的树是否满足二分搜索树的性质，这里采用中序遍历的结果来进行验证，因为中序遍历的输出
        结果是严格升序的
        Returns:
            满足返回True，否则返回False
        """
        def Inorder(node, elems_record):
            """
            对以node为根节点的二分搜索树进行中序遍历，同时将遍历的结果存储进elems_record中
            Params:
                - node: 传入的根节点
                - elems_record: 记录中序遍历结果的数组
            """
            if node is None:
                return
            
            Inorder(node.left, elems_record)
            elems_record.append(node.elem)
            Inorder(node.right, elems_record)
        
        records = []
        Inorder(self.root, records)
        # 检查records
        for i in range(len(records) - 1):
            if records[i] > records[i + 1]:
                return False
        return True
    
    def isBalanced(self):
        """
        检验当前的树是否满足平衡二叉树的性质
        Returns:
            满足返回True，否则返回False
        """        
        return self._isBananced(self.root)
        
    def getSize(self):
        """获取元素数目"""
        return self.size
    
    def isEmpty(self):
        """判空"""
        return self.size == 0
    
    def check_empty(self):
        """
        检查当前二分搜索树是否为空
        Returns:
            空就报错
        """
        if self.isEmpty():
            raise Exception('Empty queue!')
    
    def add(self, new_elem):
        """
        向二分搜索树插入元素elem
        O(logn)
        Params:
            - elem: 待插入的元素
        """
        # 返回插入元素后的新的根节点
        self.root = self._add(self.root, new_elem)
    
    def contains(self, elem):
        """
        查看某一元素是否存在于bst中
        O(logn)
        Returns:
            存在返回True,否则为False
        """
        return self._contains(self.root, elem)
    
    def preOrder(self):
        """
        二分搜索树的前序遍历
        O(n)
        前序遍历、中序遍历以及后续遍历是针对当前的根节点来说的。前序就是把对根节点的操作放在遍历左、右子树的前面，相应的中序遍历以及后序遍历以此类推
        前序遍历是最自然也是最常用的二叉搜索树的遍历方式
        """
        self._preOrder(self.root)
        
    def preOrder_nr(self):
        """
        前序遍历的非递归写法
        此时需要借助一个辅助的数据结构————栈
        O(n)
        技巧：压栈的时候先右孩子，再左孩子，从而左孩子先出栈。
        """
        # 前序遍历的非递归写法
        if self.isEmpty():
            return
        stack = [self.root]
        while len(stack):
            tmp_node = stack.pop()
            print(tmp_node.elem, end=' ')
            if tmp_node.right:
                stack.append(tmp_node.right)
            if tmp_node.left:
                stack.append(tmp_node.left)
        
    def inOrder(self):
        """
        二分搜索树的中序遍历
        O(n)
        特点：输出的元素是从小到大排列的，因为先处理左子树，到底后再处理当前节点，最后再处理右子树，而左子树的值都比当前节点小，
              右子树的值都比当前节点大，所以是排序输出
        """
        # 输出结果从小到大排列
        self._inOrder(self.root)
        
    def postOrder(self):
        """
        二分搜索树的后序遍历
        应用场景：二叉搜索树的内存回收，例如C++中的析构函数
        O(n)
        """
        # 用于析构函数，内存释放等
        self._postOrder(self.root)
        
    def levelOrder(self):
        """
        层序遍历（广度优先遍历）
        O(n)
        常用于算法设计中--无权图最短路径
        """
        if self.isEmpty():
            return
        d = deque()
        d.append(self.root)
        while len(d):
            tmp_node = d.popleft()
            print(tmp_node.elem, end=' ')
            if tmp_node.left:
                d.append(tmp_node.left)
            if tmp_node.right:
                d.append(tmp_node.right)
                
    def minimum(self):
        """
        返回当前二叉搜索树的最小值
        O(n)
        Returns:
            当前树中的最小值
        """
        self.check_empty()
        return self._minimum(self.root).elem
    
    def maximum(self):
        """
        返回当前二叉搜索树的最大值
        O(logn)
        Returns:
            当前树中的最大值
        """ 
        self.check_empty()
        return self._maximum(self.root).elem
    
    def removeMin(self):
        """
        删除当前二叉搜索树的最小值的节点
        O(logn)
        Returns: 
            被删除节点所携带的元素的值
        """
        self.check_empty()
        self.root = self._removeMin(self.root)
    
    def removeMax(self):
        """
        删除当前二叉搜索树的最大值的节点
        O(logn)
        Returns: 
            被删除节点所携带的元素的值
        """
        self.check_empty()
        self.root = self._removeMax(self.root)
        
    def remove(self, elem):
        """
        删除二叉搜索树中值为elem的节点，注意我们的二叉搜索树中的元素的值是不重复的，所以删除就是真正的删除，无残余
        这个算法是二叉搜索树中最难的一个算法
        时间复杂度：O(logn)
        Params:
            - elem: 待删除的元素
        """
        self.check_empty()
        self.root = self._remove(self.root, elem)
    
    
    # private
    def _getHeight(self, node):
        """
        获取一个节点的高度值
        Params:
            - node: 传入的节点
        """
        if node is None:  # 空节点的高度为0
            return 0
        return node.height
    
    def _getBalanceFactor(self, node):
        """
        获取某个节点的平衡因子，这里我是用左子树的高度减去右子树的高度，所以小于0的话代表右子树比左子树高，
        大于0的话代表左子树比右子树高
        Params:
            - node: 传入的节点
        """
        if node is None:
            return 0
        return self._getHeight(node.left) - self._getHeight(node.right)

    def _check_and_to_balanced(self, node):
        """
        对传入的节点进行平衡因子的检查，如果不平衡，则将其通过特定的方法转成平衡树。如果平衡，什么都不做，直接返回当前节点。
        Params:
            - node: 传入的节点
        Returns:
            平衡维护后的新的根节点，或者它本身
        """
        balanced_factor = self._getBalanceFactor(node)

        # 下面就是维护平衡性的常规4种操作了
        # LL
        if balanced_factor > 1 and self._getBalanceFactor(node.left) >= 0:
            return self._rightRotate(node)
        # RR
        elif balanced_factor < -1 and self._getBalanceFactor(node.right) <= 0:
            return self._leftRotate(node)
        # LR
        elif balanced_factor > 1 and self._getBalanceFactor(node.left) < 0:
            node.left = self._leftRotate(node.left)
            return self._rightRotate(node)
        # RL
        elif balanced_factor < -1 and self._getBalanceFactor(node.right) > 0:
            node.right = self._rightRotate(node.right)
            return self._leftRotate(node)
        else:  # 此时不用对node进行旋转操作，直接返回
            return node

    def _add(self, node, new_elem):
        """
        向以Node为根的二分搜索树插入元素elem，递归算法，这个根可以是任意节点哦，因为二分搜索树的每一个节点都是一个新的二分搜索树的根节点
        O(logn)
        Params:
            - Node: 根节点
            - elem: 带插入元素
        Returns:
            插入新节点后二分搜索树的根
        """
        if node is None:
            self.size += 1
            return Node(new_elem)
        
        if node.elem < new_elem:
            node.right = self._add(node.right, new_elem)
        elif new_elem < node.elem:
            node.left = self._add(node.left, new_elem)
        # 如果相等，我们什么都不做，即不包含重复元素
        
        # 回归的过程中需要对当前node的height属性进行更新
        node.height = max(self._getHeight(node.left), self._getHeight(node.right)) + 1
        
        # 调用check_and_to_balanced成员函数，返回新的根节点
        return self._check_and_to_balanced(node)
    
    def _contains(self, node, elem):
        """
        在以node为根的二叉搜索树中查询是否包含元素elem
        O(logn)
        Params:
            - node:    根节点
            - elem:    带查找元素
        Returns:
            bool值，存在为True
        """
        if node is None:
            return False
        
        if node.elem < elem:
            return self._contains(node.right, elem)
        elif elem < node.elem:
            return self._contains(node.left, elem)
        else:
            return True
        
    def _preOrder(self, node):
        """
        对以node为根的节点的二叉搜索树的前序遍历
        O(n)
        Params:
            - node: 当前根节点
        """
        if node is None:
            return 
        print(node.elem, end=' ')
        self._preOrder(node.left)
        self._preOrder(node.right)
        return
    
    def _inOrder(self, node):
        """
        对以node为根节点的二叉搜索树的中序遍历
        O(n)
        Params:
            - node: 当前根节点
        """
        if node is None:
            return 
        self._inOrder(node.left)
        print(node.elem, end=' ')
        self._inOrder(node.right)
        
    def _postOrder(self, node):
        """
        对以node为根节点的二叉搜索树的后序遍历
        O(n)
        Params:
            - node: 当前根节点
        """
        if node is None:
            return
        self._postOrder(node.left)
        self._postOrder(node.right)
        print(node.elem, end=' ')
        
    def _minimum(self, node):
        """
        返回以node为根的二叉搜索树携带最小值的节点
        O(logn)
        """
        if node.left is None:
            return node
        return self._minimum(node.left)
            
    def _maximum(self, node):
        """
        返回以node为根的二叉搜索树携带最大值的节点
        O(logn)
        """
        if node.right is None:
            return node
        return self._maximum(node.right)
    
    def _removeMin(self, node):
        """
        删除以node为根节点的二叉搜索树携带最小值的节点
        O(logn)
        Returns: 
            删除后的二叉搜索树的根节点，与添加操作有异曲同工之处
        """
        retNode = None
        if node.left is None:
            tmp_node = node.right
            node.right = None
            self.size -= 1
            return tmp_node
        node.left = self._removeMin(node.left)
        retNode = node
        if retNode is None:
            return 
        retNode.height = max(self._getHeight(retNode.left), self._getHeight(retNode.right)) + 1
        return self._check_and_to_balanced(retNode)
    
    def _removeMax(self, node):
        """
        删除以node为根节点的二叉搜索树携带最大值的节点
        O(logn)
        Returns: 
            删除后的二叉搜索树的根节点，与添加操作有异曲同工之处
        """
        retNode = None
        if node.right is None:
            tmp_node = node.left
            node.left = None
            self.size -= 1
            return tmp_node
        node.right = self._removeMax(node.right)
        retNode = node
        if retNode is None:
            return 
        # 回归的过程中维护树的平衡性
        retNode.height = max(self._getHeight(retNode.left), self._getHeight(retNode.right)) + 1
        return self._check_and_to_balanced(retNode)
    
    def _remove(self, node, elem):
        """
        删除以node为根节点的二叉搜索树中携带值为elem的节点
        不存在的话什么也不做
        O(logn)
        Returns: 
            删除节点后的二叉搜索树的根节点
        """
        if node is None:
            return 

        retNode = None
        if node.elem < elem:
            node.right = self._remove(node.right, elem)
            retNode = node
        elif elem < node.elem:
            node.left = self._remove(node.left, elem)
            retNode = node
        else:
            if node.left is None:
                tmp_node = node.right
                node.right = None
                self.size -= 1
                retNode = tmp_node
            elif node.right is None:
                tmp_node = node.left
                node.left = None
                self.size -= 1
                retNode = tmp_node
            else:
                # 这里采用node的后继节点
                successor = self._minimum(node.right)
                successor.right = self._removeMin(node.right)
                self.size += 1 # 要被真正删除的是node,此时已经将successor删除了，所以要补回来1个size
                successor.left = node.left
                node.left = node.right = None
                self.size -= 1
                retNode = successor

        if retNode is None:
            return 
        # 回归的过程中更新节点的高度属性
        retNode.height = max(self._getHeight(retNode.left), self._getHeight(retNode.right)) + 1

        # 回归的过程中维护树的平衡性
        return self._check_and_to_balanced(retNode)

    def _isBananced(self, node):
        """
        检验以node为根的二叉树是否满足平衡二叉树的性质
        Params:
            - node: 传入的根节点
        """
        if node is None:
            return True
        
        balanced_factor = self._getBalanceFactor(node)
        if abs(balanced_factor) > 1:
            return False
        return self._isBananced(node.left) and self._isBananced(node.right)
            
    def _rightRotate(self, y):
        """
                 y                                x
               /   \                           /    \
              x    T4                         z       y
             / \            右旋操作          /  \    / \
            z   T3         --------->       T1  T2  T3  T4 
           / \
          T1  T2
          
        对某一节点进行右旋操作，适用情况为不平衡节点在当前节点的左侧的左侧
        Params:
          - y: 传入的不平衡节点，注意y的所有孩子节点此时都是满足平衡二叉树的性质的 
        Returns:
            平衡后的新的根节点
        """
        x = y.left
        T3 = x.right
        
        # 右旋转
        x.right = y
        y.left = T3
        
        # 更新height，先更新y，再x，因为x的高度和y的高度有关
        y.height = max(self._getHeight(y.left), self._getHeight(y.right)) + 1
        x.height = max(self._getHeight(x.left), self._getHeight(x.right)) + 1
        
        return x
    
    def _leftRotate(self, y):
        """
               y                                 x
             /   \                             /   \
            T4    x           左旋操作         y     z
                 / \         --------->     /  \   / \
                T3  z                      T4  T3 T2  T1
                   / \
                  T2  T1  
                  
        对某一节点进行左旋操作，适用情况为不平衡节点在当前节点的右侧的右侧
        Params:
          - y: 传入的不平衡节点，注意y的所有孩子节点此时都是满足平衡二叉树的性质的 
        Returns:
            平衡后的新的根节点
        """
        x = y.right
        T3 = x.left
        
        # 左旋转
        x.left = y
        y.right = T3
        
        # 更新height
        y.height = max(self._getHeight(y.left), self._getHeight(y.right)) + 1
        x.height = max(self._getHeight(x.left), self._getHeight(x.right)) + 1
        
        return x

In [4]:
# test avl_tree 
def test():
    test_avl = AvlTree()
    op_nums = 1000   # 1000次测试操作
    epoches = 5  # 共随机测试5轮
    for epoch in range(epoches):  
        print('第 {}/{} 轮测试：'.format(epoch + 1, 5))
        add_set = set(np.random.randint(10000) for i in range(op_nums))
        for elem in add_set:
            test_avl.add(elem)  # 大约1000次添加操作，因为剃掉了重复元素
        assert len(add_set) == test_avl.size, 'There has some problems in "add" function\
            the set size is {}, but the tree size is {}'.format(len(add_set), test_avl.size)
        print('元素添加完毕，下面进行验证：')
        print('size: {}'.format(test_avl.getSize()))
        print('是否满足二分搜索树的性质：{}'.format(test_avl.isBst()))
        print('是否满足平衡二叉树的性质：{}'.format(test_avl.isBalanced()))

        print('下面测试avl树的删除节点的操作的正确性,将添加的元素全部删除：')
        for elem in add_set:
            test_avl.remove(elem)
            # 每次删除完一个元素我们都验证一下
            if not test_avl.isBst(): 
                raise Exception('在删除 {} 元素的时候，此时avl树不满足二分搜索树的性质！'.format(elem))
            if not test_avl.isBalanced(): 
                raise Exception('在删除 {} 元素的时候，此时avl树不满足平衡二叉树的性质！'.format(elem))
        print('...')
        print('删除完毕，此时的size: {}'.format(test_avl.getSize()))
        assert test_avl.size == 0, 'There has some problems in "remove" function,\
            after the remove function, the tree size is {}, but not zero.'.format(test_avl.size)
        
        print('测试完毕，无任何错误！')
        print('-' * 30)
        
test()

第 1/5 轮测试：
元素添加完毕，下面进行验证：
size: 957
是否满足二分搜索树的性质：True
是否满足平衡二叉树的性质：True
下面测试avl树的删除节点的操作的正确性,将添加的元素全部删除：
...
删除完毕，此时的size: 0
测试完毕，无任何错误！
------------------------------
第 2/5 轮测试：
元素添加完毕，下面进行验证：
size: 955
是否满足二分搜索树的性质：True
是否满足平衡二叉树的性质：True
下面测试avl树的删除节点的操作的正确性,将添加的元素全部删除：
...
删除完毕，此时的size: 0
测试完毕，无任何错误！
------------------------------
第 3/5 轮测试：
元素添加完毕，下面进行验证：
size: 944
是否满足二分搜索树的性质：True
是否满足平衡二叉树的性质：True
下面测试avl树的删除节点的操作的正确性,将添加的元素全部删除：
...
删除完毕，此时的size: 0
测试完毕，无任何错误！
------------------------------
第 4/5 轮测试：
元素添加完毕，下面进行验证：
size: 956
是否满足二分搜索树的性质：True
是否满足平衡二叉树的性质：True
下面测试avl树的删除节点的操作的正确性,将添加的元素全部删除：
...
删除完毕，此时的size: 0
测试完毕，无任何错误！
------------------------------
第 5/5 轮测试：
元素添加完毕，下面进行验证：
size: 941
是否满足二分搜索树的性质：True
是否满足平衡二叉树的性质：True
下面测试avl树的删除节点的操作的正确性,将添加的元素全部删除：
...
删除完毕，此时的size: 0
测试完毕，无任何错误！
------------------------------


# 用AVL树实现一个集合

In [5]:
# 非常简单，和二分搜索树那一节几乎是一样的
class AvlSet:
    def __init__(self):
        """集合构造函数"""
        self.data = AvlTree()
        # data里面内置了getSize()方法，所以就不用size成员变量了
    
    def getSize(self):
        """获取集合大小"""
        return self.data.getSize()
    
    def isEmpty(self):
        """判空"""
        return self.data.isEmpty()
    
    def add(self, elem):
        """
        向集合中添加元素
        Params:
            - elem: 待添加元素
        """
        self.data.add(elem)
        
    def contains(self, elem):
        """
        判断集合中是否包含某以元素
        Params:
            - elem: 待查询元素
        Returns:
            存在返回True，否则为False
        """
        return self.data.contains(elem)
    
    def remove(self, elem):
        """
        从集合中删除一个元素，若不存在什么也不做
        Params:
            - elem: 待删除元素
        """
        self.data.remove(elem)
        
    def print_(self):
        """对集合中的元素进行打印"""
        self.data.inOrder()  # 中序遍历吧，此时从小到大排列，比较舒服，当然，其他遍历方法随意

In [6]:
# test AvlTreeSet
nums = [10, 20, 30, 30, 30, 30, 15, 26, 77]
test_avl_set = AvlSet()
print('将nums数据中的数据添加进集合-----', end=' ')
for elem in nums:
    test_avl_set.add(elem)
test_avl_set.print_()
print('\n集合的size-----', test_avl_set.getSize())
print('是否包含元素77？-----', test_avl_set.contains(77))
print('将元素30从集合中移除-----', end=' ')
test_avl_set.remove(30)
test_avl_set.print_()

将nums数据中的数据添加进集合----- 10 15 20 26 30 77 
集合的size----- 6
是否包含元素77？----- True
将元素30从集合中移除----- 10 15 20 26 77 

# 用AVL树实现一个字典（映射）
- 用到的基本都是AVL树中的成员函数，非常容易实现

In [7]:
# 注意Node的key必须支持比较操作
class Node_Avlmap:
    def __init__(self, k, v):
        """
        字典中节点类的构造函数
        Params:
            k: 键
            v: 键所对应的value值
        """
        self.key = k
        self.value = v
        self.left = self.right = None
        self.height = 1

In [8]:
class Avlmap:
    def __init__(self):
        """二分搜索树的构造函数"""
        self.root = None
        self.size = 0
        
    def getSize(self):
        """获取当前字典的大小"""
        return self.size
    
    def isEmpty(self):
        """判空"""
        return self.size == 0
    
    def check_empty(self):
        """
        检查当前二分搜索树是否为空
        Returns:
            空就报错
        """
        if self.isEmpty():
            raise Exception('Empty queue!')
    
    def add(self, k, v):
        """
        向字典中添加某一键值对
        Params:
            - k: 新的键
            - v: 新的键所对应的值
        """
        # 返回插入元素后的新的根节点
        self.root = self._add(self.root, k, v)
    
    def contains(self, k):
        """
        查询当前的字典中是否包含键k
        Params:
            k: 待查询的键
        Returns:
            存在返回True，否则返回False
        """
        return self._getnode(self.root, k) is not None 

    def set(self, k, v):
        """
        将字典中键为k的键值对的值设为新值
        Params:
            - k: 待设定的key
            - v: 新的value
        """
        # 此时用户已经明确的直到了键k存在于字典中，否则报错
        dst_node = self._getnode(self.root, k)
        if not dst_node:
            raise Exception('The key:{} is not in the map!'.format(k))
        dst_node.value = v

    def get(self, k):
        """
        获取字典中键为k的value，不存在则返回None
        Params:
            - k: 输入的key值
        """
        dst_node = self._getnode(self.root, k)
        if not dst_node:
            return None
        return dst_node.value

    def minimum(self):
        """
        获取字典中最小的键及其value
        Returns:
            一个tuple，第一个元素为键，第二个元素为键所对应的value
        """
        self.check_empty()
        ret_node = self._minimum(self.root)
        return ret_node.key, ret_node.value

    def removeMin(self):
        """删除字典中键最小的那个键值对，并对字典中的根节点进行相应的更新"""
        self.check_empty()
        self.root = self._removeMin(self.root)
        
    def remove(self, k):
        """
        删除字典中键为k的键值对，并对字典中的根节点进行相应的更新，若不存在什么都不做
        Params:
            - k: 待删除键值对的key
        """
        self.check_empty()
        self.root = self._remove(self.root, k)

    def print_(self):
        """打印字典中的所有元素，这里我采用广度优先遍历来打印字典了"""
        print('[', end=' ')
        if self.root is None:
            return 
        d = deque()
        d.append(self.root)
        while len(d):
            tmp_node = d.popleft()
            print('{}-->{}'.format(tmp_node.key, tmp_node.value), end=', ')
            if tmp_node.left:
                d.append(tmp_node.left)
            if tmp_node.right:
                d.append(tmp_node.right)
        print(']')

    
    # private
    def _getHeight(self, node):
        """
        获取一个节点的高度值
        Params:
            - node: 传入的节点
        """
        if node is None:  # 空节点的高度为0
            return 0
        return node.height
    
    def _getBalanceFactor(self, node):
        """
        获取某个节点的平衡因子，这里我是用左子树的高度减去右子树的高度，所以小于0的话代表右子树比左子树高，
        大于0的话代表左子树比右子树高
        Params:
            - node: 传入的节点
        """
        if node is None:
            return 0
        return self._getHeight(node.left) - self._getHeight(node.right)

    def _check_and_to_balanced(self, node):
        """
        对传入的节点进行平衡因子的检查，如果不平衡，则将其通过特定的方法转成平衡树。如果平衡，什么都不做，直接返回当前节点。
        Params:
            - node: 传入的节点
        Returns:
            平衡维护后的新的根节点，或者它本身
        """
        balanced_factor = self._getBalanceFactor(node)

        # 下面就是维护平衡性的常规4种操作了
        # LL
        if balanced_factor > 1 and self._getBalanceFactor(node.left) >= 0:
            return self._rightRotate(node)
        # RR
        elif balanced_factor < -1 and self._getBalanceFactor(node.right) <= 0:
            return self._leftRotate(node)
        # LR
        elif balanced_factor > 1 and self._getBalanceFactor(node.left) < 0:
            node.left = self._leftRotate(node.left)
            return self._rightRotate(node)
        # RL
        elif balanced_factor < -1 and self._getBalanceFactor(node.right) > 0:
            node.right = self._rightRotate(node.right)
            return self._leftRotate(node)
        else:  # 此时不用对node进行旋转操作，直接返回
            return node

    def _add(self, node, k, v):
        """
        向以node为根的字典中添加键值对，如果键已经存在，则视为对输入的键对应的value进行更新操作
        Params:
            - k: 待添加键值对的key
            - v: 待添加键值对的value
        Returns:
            新的根节点
        """
        if node is None:
            self.size += 1
            return Node_Avlmap(k, v)
        
        if node.key < k:
            node.right = self._add(node.right, k, v)
        elif k < node.key:
            node.left = self._add(node.left, k, v)
        else:
            node.value = v
        
        # 回归的过程中需要对当前node的height属性进行更新
        node.height = max(self._getHeight(node.left), self._getHeight(node.right)) + 1
        
        # 调用check_and_to_balanced成员函数，返回新的根节点
        return self._check_and_to_balanced(node)
        
    def _minimum(self, node):
        """
        返回以node为根的二叉搜索树携带最小值的节点
        O(logn)
        """
        if node.left is None:
            return node
        return self._minimum(node.left)
    
    def _removeMin(self, node):
        """
        删除以node为根节点的二叉搜索树携带最小值的节点
        O(logn)
        Returns: 
            删除后的二叉搜索树的根节点，与添加操作有异曲同工之处
        """
        retNode = None
        if node.left is None:
            tmp_node = node.right
            node.right = None
            self.size -= 1
            return tmp_node
        node.left = self._removeMin(node.left)
        retNode = node
        if retNode is None:
            return 
        retNode.height = max(self._getHeight(retNode.left), self._getHeight(retNode.right)) + 1
        return self._check_and_to_balanced(retNode)
    
    def _remove(self, node, k):
        """
        删除以node为根节点的字典键为k的节点
        Params:
            - node: 输入的根节点
            - k: 待删除的key
        Returns:
            新的根节点
        """
        if node is None:
            return 

        retNode = None
        if node.key < k:
            node.right = self._remove(node.right, k)
            retNode = node
        elif k < node.key:
            node.left = self._remove(node.left, k)
            retNode = node
        else:
            if node.left is None:
                tmp_node = node.right
                node.right = None
                self.size -= 1
                retNode = tmp_node
            elif node.right is None:
                tmp_node = node.left
                node.left = None
                self.size -= 1
                retNode = tmp_node
            else:
                # 这里采用node的后继节点
                successor = self._minimum(node.right)
                successor.right = self._removeMin(node.right)
                self.size += 1 # 要被真正删除的是node,此时已经将successor删除了，所以要补回来1个size
                successor.left = node.left
                node.left = node.right = None
                self.size -= 1
                retNode = successor

        if retNode is None:
            return 
        # 回归的过程中更新节点的高度属性
        retNode.height = max(self._getHeight(retNode.left), self._getHeight(retNode.right)) + 1

        # 回归的过程中维护树的平衡性
        return self._check_and_to_balanced(retNode)

    def _isBananced(self, node):
        """
        检验以node为根的二叉树是否满足平衡二叉树的性质
        Params:
            - node: 传入的根节点
        """
        if node is None:
            return True
        
        balanced_factor = self._getBalanceFactor(node)
        if abs(balanced_factor) > 1:
            return False
        return self._isBananced(node.left) and self._isBananced(node.right)

    def _getnode(self, node, k):
        """
        根据某一个key在以node为根节点的字典中寻找到相应的节点
        Params:
            - node: 输入的根节点
            - k: 待查询的key
        Returns:
            返回携带对应键的Node，若不存在返回None
        """
        if node is None:
            return None 

        if node.key < k:
            return self._getnode(node.right, k)
        elif k < node.key:
            return self._getnode(node.left, k)
        else:
            return node 
            
    def _rightRotate(self, y):
        """
                 y                                x
               /   \                           /    \
              x    T4                         z       y
             / \            右旋操作          /  \    / \
            z   T3         --------->       T1  T2  T3  T4 
           / \
          T1  T2
          
        对某一节点进行右旋操作，适用情况为不平衡节点在当前节点的左侧的左侧
        Params:
          - y: 传入的不平衡节点，注意y的所有孩子节点此时都是满足平衡二叉树的性质的 
        Returns:
            平衡后的新的根节点
        """
        x = y.left
        T3 = x.right
        
        # 右旋转
        x.right = y
        y.left = T3
        
        # 更新height，先更新y，再x，因为x的高度和y的高度有关
        y.height = max(self._getHeight(y.left), self._getHeight(y.right)) + 1
        x.height = max(self._getHeight(x.left), self._getHeight(x.right)) + 1
        
        return x
    
    def _leftRotate(self, y):
        """
               y                                 x
             /   \                             /   \
            T4    x           左旋操作         y     z
                 / \         --------->     /  \   / \
                T3  z                      T4  T3 T2  T1
                   / \
                  T2  T1  
                  
        对某一节点进行左旋操作，适用情况为不平衡节点在当前节点的右侧的右侧
        Params:
          - y: 传入的不平衡节点，注意y的所有孩子节点此时都是满足平衡二叉树的性质的 
        Returns:
            平衡后的新的根节点
        """
        x = y.right
        T3 = x.left
        
        # 左旋转
        x.left = y
        y.right = T3
        
        # 更新height
        y.height = max(self._getHeight(y.left), self._getHeight(y.right)) + 1
        x.height = max(self._getHeight(x.left), self._getHeight(x.right)) + 1
        
        return x

In [9]:
# test AvlMap
random.seed(7)
test_map = Avlmap()
key_nums = [i for i in range(1, 10)]
random.shuffle(key_nums)
value_nums = [chr(i) for i in range(65, 73)]
random.shuffle(value_nums)
nums = [(i, j) for (i, j) in zip(key_nums, value_nums)]
random.shuffle(nums)
print('待添加的key,value-----', nums)
print('将它们添加进字典中-----', end=' ')
#              3(F) 
#           /      \
#       1(G)       8(H)
#        \         /   \ 
#        2(D)     5(B)  9(A)
#                /  \
#              4(E) 7(C)
for elem in nums:
    test_map.add(*elem)
test_map.print_()
print('用add方法对已存在的键相应的值进行更新，将7的值设为666-----', end=' ')
test_map.add(7, 666)
test_map.print_()
print('将键为7的键值对从字典中删除-----', end=' ')
test_map.remove(7)
test_map.print_()
print('将键为8的值设为"hahaha"-----', end=' ')
test_map.set(8, 'hahaha')
test_map.print_()
print('获取键为8的相应的值-----', test_map.get(8))
print('此时的size-----', test_map.getSize())
print('最小的键为-----', test_map.minimum()[0])
print('将最小的键所在的键值对删除-----', end=' ')
test_map.removeMin()
test_map.print_()
print('此时字典中是否包含键为9的键值对？-----', test_map.contains(9))

待添加的key,value----- [(3, 'F'), (1, 'G'), (8, 'H'), (9, 'A'), (7, 'C'), (2, 'D'), (5, 'B'), (4, 'E')]
将它们添加进字典中----- [ 3-->F, 1-->G, 8-->H, 2-->D, 5-->B, 9-->A, 4-->E, 7-->C, ]
用add方法对已存在的键相应的值进行更新，将7的值设为666----- [ 3-->F, 1-->G, 8-->H, 2-->D, 5-->B, 9-->A, 4-->E, 7-->666, ]
将键为7的键值对从字典中删除----- [ 3-->F, 1-->G, 8-->H, 2-->D, 5-->B, 9-->A, 4-->E, ]
将键为8的值设为"hahaha"----- [ 3-->F, 1-->G, 8-->hahaha, 2-->D, 5-->B, 9-->A, 4-->E, ]
获取键为8的相应的值----- hahaha
此时的size----- 7
最小的键为----- 1
将最小的键所在的键值对删除----- [ 5-->B, 3-->F, 8-->hahaha, 2-->D, 4-->E, 9-->A, ]
此时字典中是否包含键为9的键值对？----- True
